In [ ]:
import pandas as pd

# Load your data (adjust path if needed)
data_full = pd.read_csv('../merged_data/full_info_dataframe.csv', sep=",")

In [8]:
# Filter to only rows where process_step is 'confirm'
confirm_visits = data_full[data_full['process_step'] == 'confirm']

# Drop duplicates to ensure each visitor is counted once
confirm_unique_visitors = confirm_visits[['visitor_id', 'Variation']].drop_duplicates()

# Group by Variation and count
finished_counts = confirm_unique_visitors.groupby('Variation')['visitor_id'].nunique()

print(finished_counts)

Variation
Control    15566
Test       19505
Name: visitor_id, dtype: int64
